In [2]:
# !py -m pip install opencv-python
# !python.exe -m pip install --upgrade pip
import numpy as np
import cv2 as cv
from numpy import linalg
from math import acos, sqrt, degrees
from matplotlib import pyplot as plt

# odabir prvih levih tacaka

In [45]:
leve8 = []
brojac = 0
def click_event(event, x, y, flags, params): 
    global brojac
    font = cv.FONT_HERSHEY_SIMPLEX 

    if event == cv.EVENT_LBUTTONDOWN: 
        cv.putText(slika, f'P{brojac+1}(' + str(x) + ',' +
                    str(y) + ')', (x,y), font, 
                    1, (0, 0, 0), 2) 
        cv.imshow("lijeva", slika)
        
        leve8.append([x, y, 1])
        brojac += 1
        
        if brojac == 16:
            cv.imwrite('lijevaOznacena.jpg', slika)
            cv.imshow("lijeva", slika)

In [46]:
putanja = './lijeva.jpg'
slika = cv.imread(putanja)

cv.imshow("lijeva", slika)

cv.setMouseCallback("lijeva", click_event)

cv.waitKey(0)
cv.destroyAllWindows()

In [47]:
np.array(leve8)

array([[ 347,  642,    1],
       [ 559,  714,    1],
       [ 423,  829,    1],
       [ 181,  763,    1],
       [ 774,  793,    1],
       [1037,  948,    1],
       [ 975, 1003,    1],
       [ 697,  836,    1]])

# odabir prvih desnih tacaka

In [48]:
desne8 = []
brojac = 0
def click_event(event, x, y, flags, params): 
    global brojac
    font = cv.FONT_HERSHEY_SIMPLEX 

    if event == cv.EVENT_LBUTTONDOWN: 
        cv.putText(slika, f'P{brojac+1}(' + str(x) + ',' +
                    str(y) + ')', (x,y), font, 
                    1, (0, 0, 0), 2) 
        cv.imshow("desna", slika)
        
        desne8.append([x, y, 1])
        brojac += 1
        
        if brojac == 16:
            cv.imwrite('desnaOZnacena.jpg', slika)
            cv.imshow("desna", slika)

In [49]:
putanja = './desna.jpg'
slika = cv.imread(putanja)

cv.imshow("desna", slika)

cv.setMouseCallback("desna", click_event)

cv.waitKey(0)
cv.destroyAllWindows()

In [50]:
np.array(desne8)

array([[ 394,  648,    1],
       [ 556,  722,    1],
       [ 352,  820,    1],
       [ 199,  748,    1],
       [ 838,  826,    1],
       [1071, 1014,    1],
       [ 955, 1064,    1],
       [ 733,  864,    1]])

In [51]:
import numpy
import numpy as np
import math
import plotly.graph_objects as go
import plotly.express as px

In [52]:
def piksel(koord): #Odredjivanje piksel koordinata tacke
    return np.array([1,-1,-1])*(np.array([1200,0,0])-koord) #promeni 1600 za sliku

In [53]:
def fundJed(xl,xd): #Odredjivanje jednacina za fundamentalnu matricu
    return np.array([xl[0]*xd[0],xl[1]*xd[0],xl[2]*xd[0],xl[0]*xd[1],xl[1]*xd[1],xl[2]*xd[1],xl[0]*xd[2],xl[1]*xd[2],xl[2]*xd[2]])

In [54]:
def fundamentalnaMatrica(leve,desne): #Odredjivanje fundamentalne matrice f
    a=[]
    for x,xp in zip(leve,desne):
        jed= fundJed(x,xp)
        a.append(jed)
    a = np.array(a)
    U, D, V = np.linalg.svd(a)
    f = V[-1]
    f = f.reshape(3,3)
    return f

In [55]:
def skew2v(EC): #Uzima vektor iz kososimetricne matrice 
    return np.array([EC[2][1],EC[0][2],EC[1][0]])

In [56]:
def dekomponuj(e): #Dekompozicija osnove matrice E
    q0=np.array([[0,-1,0],[1,0,0],[0,0,1]])
    e0=np.array([[0,1,0],[-1,0,0],[0,0,0]])
    U,S,V=np.linalg.svd(e)
    EC=U@e0@np.transpose(U)
    A=U@q0@V #probaj verziju bez transponovanih q0,v ako nesto ne valja
    return A,skew2v(EC) #skew2v mozda vrati suprotan znak, stavi mu minus ako ne valja

In [57]:
def kamera1(e): #Matrica kamere u koordinatnom sistemu druge kamere
    A,C=dekomponuj(e)
    C=K1 @ -np.transpose(A) @ C
    C=-C
    A=K1 @ np.transpose(A)
    A1=np.concatenate((np.transpose(A),[C]),axis=0) #append A.T,C
    return np.transpose(A1)

In [58]:
def triangulisi(T1,T2,pt1,pt2): #Triangulacija tacaka sa fotografije
    a=np.array([pt1[1]*T1[2,:]-pt1[2]*T1[1,:],
                -pt1[0]*T1[2,:]+pt1[2]*T1[0,:],
                pt2[1]*T2[2,:]-pt2[2]*T2[1,:],
                -pt2[0]*T2[2,:]+pt2[2]*T2[0,:]]) #ima na slajdovima
    U,S,V=np.linalg.svd(a)
    pt=V[3] # valjda moze -1
    pt=(1/pt[3])*pt
    pt=np.where(pt==0,0.0,pt)
    return np.array(pt)

In [59]:
def recPts(c1,c2,pts1,pts2):
    a=[]
    for x,xp in zip(pts1,pts2):
        X=triangulisi(c1,c2,x,xp)
        X=np.delete(X,-1)
        a.append(X)
    a = np.array(a)
    return a

In [60]:
K1=np.array(
        [[1400,0,600],
         [0,1400,800],
         [0,0,1]]
        )
kam2=np.array(
        [[1400,0,600,0],
         [0,1400,800,0],
         [0,0,1,0]]
        )

# odabir ostalih levih tacaka

In [61]:
leve16 = []
brojac = 0
def click_event(event, x, y, flags, params): 
    global brojac
    font = cv.FONT_HERSHEY_SIMPLEX 

    if event == cv.EVENT_LBUTTONDOWN: 
        cv.putText(slika, f'P{brojac+1}(' + str(x) + ',' +
                    str(y) + ')', (x,y), font, 
                    1, (0, 0, 0), 2) 
        cv.imshow("leva", slika)
        
        leve16.append([x, y, 1])
        brojac += 1

In [62]:
putanja = './lijeva.jpg'
slika = cv.imread(putanja)

cv.imshow("leva", slika)

cv.setMouseCallback("leva", click_event)

cv.waitKey(0)
cv.destroyAllWindows()

# odabir ostalih desnih tacaka

In [63]:
desne16 = []
brojac = 0
def click_event(event, x, y, flags, params): 
    global brojac
    font = cv.FONT_HERSHEY_SIMPLEX 

    if event == cv.EVENT_LBUTTONDOWN: 
        cv.putText(slika, f'P{brojac+1}(' + str(x) + ',' +
                    str(y) + ')', (x,y), font, 
                    1, (0, 0, 0), 2) 
        cv.imshow("desna", slika)
        
        desne16.append([x, y, 1])
        brojac += 1
        
        if brojac == 16:
            cv.imwrite('desnaOZnacena.jpg', slika)
            cv.imshow("desna", slika)

In [64]:
putanja = './desna.jpg'
slika = cv.imread(putanja)

cv.imshow("desna", slika)

cv.setMouseCallback("desna", click_event)

cv.waitKey(0)
cv.destroyAllWindows()

In [65]:
f=fundamentalnaMatrica(piksel(leve8),piksel(desne8))
#osnovna matrica
e=np.transpose(K1)@f@K1
pts3D=recPts(kamera1(e),kam2,piksel(leve16),piksel(desne16))
print(pts3D)

[[ 0.13001229 -0.22304598  1.10430739]
 [ 0.00518623 -0.11434872  0.82442751]
 [ 0.16315093 -0.06790751  1.0225691 ]
 [ 0.39489644 -0.18965548  1.49762399]
 [ 0.11927964 -0.03513555  1.06439657]
 [ 0.0061648   0.01812965  0.82280017]
 [ 0.14371909  0.10596518  1.00259304]
 [ 0.31777888  0.02918768  1.35620637]
 [-0.11787571 -0.02984011  0.62168121]
 [-0.16129687  0.05270288  0.46747552]
 [-0.13069897  0.07611288  0.50740655]
 [-0.07598155 -0.01271492  0.68243607]
 [-0.11491672 -0.00936817  0.62381108]
 [-0.16087988  0.07106867  0.4715682 ]
 [-0.12743848  0.0959344   0.51244883]
 [-0.07480251  0.00872514  0.68614342]]


In [66]:
print(f)

[[ 2.43769033e-07 -6.22651983e-07  6.69780099e-04]
 [ 1.63337657e-06  7.39139121e-07  7.69702819e-04]
 [-1.63498586e-03 -2.64750432e-03  9.99994638e-01]]


In [67]:
print(e)

[[ 0.4777873  -1.22039789  0.44508791]
 [ 3.20141809  1.44871268  3.27745609]
 [-0.25483245 -3.4016979  -0.03541633]]


In [68]:
A, E0 = dekomponuj(e)
print(A)

[[ 0.40074895 -0.23757266 -0.88484999]
 [-0.00597609 -0.9664526   0.25677551]
 [-0.91616842 -0.09761457 -0.38872462]]


In [69]:
print("E0")
print(E0)

E0
[ 0.90995577 -0.14659824 -0.38792969]


In [70]:
print("T1:")
print(kamera1(e))

T1:
[[ 3.01385320e+01  1.45698779e+02 -1.51587055e+03  5.94116746e+02]
 [-1.04048172e+03 -1.14761323e+03 -4.47640094e+02 -6.04901382e+02]
 [-8.84849993e-01  2.56775509e-01 -3.88724618e-01 -6.92019375e-01]]


In [71]:
print("T2:")
print(kam2)

T2:
[[1400    0  600    0]
 [   0 1400  800    0]
 [   0    0    1    0]]


In [75]:
import plotly.graph_objects as go
import plotly.express as px

# U ovom slucaju crtamo jednostavnu kocku. Vi cete koristiti rekonstruisana temena
temenaKocke = pts3D #np.array([[0,0, 3], [0, 3,3], [3,3,3],[3, 0,3],[0,0,0], [0, 3,0], [3, 3,0],[3,0,0]] )
# Ovo su ivice kocke zadate indeksima temena
ivice = [[0, 1], 
         [0, 3], 
         [0, 4], 
         [2, 3], 
         [2, 1], 
         [2, 6], 
         [5, 6], 
         [5, 4], 
         [5, 1], 
         [7, 6],
         [7, 3], 
         [7, 4], #do ovde je kocka
         [8, 9], 
         [8, 11],
         [8, 12], 
         [10, 11], 
         [10, 9], 
         [10, 14], 
         [13, 14], 
         [13, 12], 
         [13, 9], 
         [15, 14],
         [15, 11], 
         [15, 12], #do ovde je kocka
        ]
# dodatne mogucnosti istrazite sami

#  prikaz scene - koristi "plotly" biblioteku
def prikazKocke(): 
    # izdvajamo x,y,z koordinate svih tacaka
    xdata = (np.transpose(temenaKocke))[0]
    ydata = (np.transpose(temenaKocke))[1]
    zdata = (np.transpose(temenaKocke))[2]
    # u data1 ubacujemo sve sto treba naccrtati
    data1 = []
    # za svaku ivicu crtamo duz na osnovu koordinata
    for i in range(len(ivice)):
        data1.append(go.Scatter3d(x=[xdata[ivice[i][0]], xdata[ivice[i][1]]], y=[ydata[ivice[i][0]], ydata[ivice[i][1]]],z=[zdata[ivice[i][0]], zdata[ivice[i][1]]]))
    fig = go.Figure(data = data1 )
    # da ne prikazuje legendu
    fig.update_layout(showlegend=False)
    # pravi html fajl (ako zelite da napravite "rotatable" 3D rekonstruciju)
    # birate kao parametar velicinu apleta. fulhtml=False je vazno da ne bi pravio ogroman fajl
    # ovde stavite neki vas folder
    fig.write_html("./rekonstrukcija", include_plotlyjs = 'cdn', default_width = '800px', default_height = '600px', full_html = False) #Modifiy the html file
    fig.show()

prikazKocke()

In [ ]:
def centar(A):
    # brisemo kolone redom i nalazimo determinante
    A1 = np.linalg.det(np.delete(A, 0, 1))
    A2 = np.linalg.det(np.delete(A, 1, 1))
    A3 = np.linalg.det(np.delete(A, 2, 1))
    A4 = np.linalg.det(np.delete(A, 3, 1))

    C = (-1/A4)*np.array([A1, -A2, A3, -A4])
    C = np.where(np.isclose(C, 0) , 0.0 , C)  # izbegavanje -0. u rezultatu
    return C

In [ ]:
def kameraA(T):
    T0 = np.delete(T, 3, 1)
    if linalg.det(T0) < 0:
        T0 = -T0
    Q, R = linalg.qr(linalg.inv(T0)) 

    if R[0][0] < 0:
        R = np.diag([-1, 1, 1])@R
        Q = Q@np.diag([-1, 1, 1])
    if R[1][1] < 0:
        R = np.diag([1, -1, 1])@R
        Q = Q@np.diag([1, -1, 1])
    if R[2][2] < 0:
        R = np.diag([1, 1, -1])@R
        Q = Q@np.diag([1, 1, -1])

    A = np.transpose(Q)
    A = np.where(np.isclose(A, 0) , 0.0 , A)  # izbegavanje -0. u rezultatu
    return A


In [ ]:
def kameraK(T):
    T0 = np.delete(T, 3, 1)
    if linalg.det(T0) < 0:
        T0 = -T0
    Q, R = linalg.qr(linalg.inv(T0)) 

    if R[0][0] < 0:
        R = np.diag([-1, 1, 1])@R
        Q = Q@np.diag([-1, 1, 1])
    if R[1][1] < 0:
        R = np.diag([1, -1, 1])@R
        Q = Q@np.diag([1, -1, 1])
    if R[2][2] < 0:
        R = np.diag([1, 1, -1])@R
        Q = Q@np.diag([1, 1, -1])

    K = linalg.inv(R)
    if K[2][2]:
        K = K/K[2][2]
    K = np.where(np.isclose(K, 0) , 0.0 , K)  # izbegavanje -0. u rezultatu
    return K

In [ ]:
def matricaKamere(pts2D, pts3D):
    A = []
    for i in range(len(pts2D)):
        x1, x2, x3 = pts2D[i]
        xp1, xp2, xp3, xp4 = pts3D[i]
        A.append([0, 0, 0, 0, -x3*xp1, -x3*xp2, -x3*xp3, -x3*xp4, x2*xp1, x2*xp2, x2*xp3, x2*xp4])
        A.append([x3*xp1, x3*xp2, x3*xp3, x3*xp4, 0, 0, 0, 0, -x1*xp1, -x1*xp2, -x1*xp3, -x1*xp4])
    A = np.array(A)

    _, _, V = linalg.svd(A)
    T = np.array(V[-1]).reshape(3, 4)
    T /= T[2][3]
    
    T = np.where(np.isclose(T, 0) , 0.0 , T)
    return T

In [ ]:
T = matricaKamere(org, img)

In [ ]:
org =  np.array([1,-1,-1])*(np.array([1600,0,0])- np.array(ts))

In [ ]:
img = np.array([[0,0,3,1], [0,1,3,1], [0,2,3,1],[0,3,3,1],
                [1,0,3,1], [1,1,3,1], [1,2,3,1],[1,3,3,1],
                [2,0,3,1], [2,1,3,1], [2,2,3,1],[2,3,3,1],
                [3,0,3,1], [3,1,3,1], [3,2,3,1],[3,3,3,1], #gornja strana
                [0,3,2,1], [1,3,2,1], [2,3,2,1],[3,3,2,1],
                [0,3,1,1], [1,3,1,1], [2,3,1,1],[3,3,1,1],
                [0,3,0,1], [1,3,0,1], [2,3,0,1],[3,3,0,1], #desna
                [3,0,2,1],[3,1,2,1],[3,2,2,1], [3,3,2,1],
                [3,0,1,1],[3,1,1,1],[3,2,1,1], [3,3,1,1],
                [3,0,0,1],[3,1,0,1],[3,2,0,1], [3,3,0,1]])

In [ ]:
print('Matrica kamere:')
print(T)
print('\nPozicija centra kamere:')
C = centar(T)
print(C)
print('\nSpoljasnja matrica kamere:')
A = kameraA(T)
print(A)
print('\nMatrica kalibracije kamere:')
K = kameraK(T)
print(K)

In [ ]:
n_voxels = np.zeros((3, 3, 3), dtype=bool)
for i in range(3):
    for j in range(3):
        for k in range(3):
            if (i+j+k)%2 == 1:
                n_voxels[i,j,k] = True
filled = np.ones(n_voxels.shape)
ax = plt.figure().add_subplot(projection='3d')
ax.voxels(filled, facecolors='yellow')

ax.quiver(0,0,0, 4,0,0, color='red', label='x', alpha=0.9)
ax.text(4.2, 0, 0, s="X")
ax.quiver(0,0,0, 0,8,0, color='green', label='y', alpha=0.9)
ax.text(0, 8.2, 0, s="Y")
ax.quiver(0,0,0, 0,0,4, color='blue', label='z', alpha=0.9)
ax.text(0, 0, 4.2, s="Z")

ax.quiver(C[0], C[1], C[2], A[0][0], A[0][1], A[0][2], color='red', label='x', alpha=0.9)
ax.quiver(C[0], C[1], C[2], A[1][0], A[1][1], A[1][2], color='green', label='y', alpha=0.9)
ax.quiver(C[0], C[1], C[2], A[2][0], A[2][1], A[2][2], color='blue', label='z', alpha=0.9)
ax.scatter(C[0], C[1], C[2])
ax.text(C[0]+0.2, C[1]+0.1, C[2]+0.8, s="C")


plt.show()

In [ ]:
def afinize(tacka):
    return list(map(lambda x : x / tacka[2], tacka))[:-1]

In [ ]:
def cross(x, y):
    return np.cross(x, y)

In [ ]:
def tacka_nedogleda(t1, t2, t3, t4):
    return list(cross(cross(t1, t2), cross(t3, t4)))

In [ ]:
def zaokruzi(tacka):
    return list(map(lambda x : round(x), tacka))

In [ ]:

# def homogenize(lista_tacaka):
#     for tacka in lista_tacaka:
#         tacka = tacka.append(1)
#     return lista_tacaka

def osmoteme(lista_tacaka):
    #pretvori u homogene
#     lista_tacaka = homogenize(lista_tacaka)
    #izracunaj_tacku
    t1, t2, t3, t5, t6, t7, t8 = lista_tacaka
    
    xb = tacka_nedogleda(t2, t6, t1, t5)
    yb = tacka_nedogleda(t5, t6, t7, t8)
    return afinize(tacka_nedogleda(t8, xb, t3, yb))

In [ ]:
ts

In [ ]:
osmoteme(ts)

In [ ]:
t1, t2, t3, t5, t6, t7, t8 = ts
    
xb = zaokruzi(tacka_nedogleda(t2, t6, t1, t5))
xb

In [ ]:
yb = zaokruzi(tacka_nedogleda(t5, t6, t7, t8))
yb

In [ ]:
tacka_nedogleda(t8, xb, t3, yb)

In [17]:
def osmoteme(ts):
    return [268, 563]